### ✅ Step 1: Load and Explore the Dataset

In [1]:
import pandas as pd

Dataset link -> https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset

In [2]:
fake_df = pd.read_csv("Data/Fake.csv")
real_df = pd.read_csv("Data/True.csv")

In [3]:
real_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [4]:
fake_df["label"] = 0
real_df["label"] = 1

In [5]:
real_df.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [6]:
data = pd.concat([fake_df, real_df], ignore_index=True)

In [7]:
data.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [8]:
data = data.sample(frac=1, random_state=7).reset_index(drop=True)

In [9]:
data["text"] = data["title"].fillna('') + " " + data["text"].fillna('')

In [10]:
data = data.drop(columns=["subject", "date", "title"])

In [11]:
print("Dataset shape:", data.shape)
print("Class distribution:\n", data["label"].value_counts())
data.head()

Dataset shape: (44898, 2)
Class distribution:
 label
0    23481
1    21417
Name: count, dtype: int64


,text,label
0,Senate backs massive increase in military spen...,1
1,"Anonymous GOP Reps Admit Clinton Will Win, Bl...",0
2,‘A Gimmick’: It Just Dawned On Republicans Th...,0
3,USA's Tillerson and Saudi crown prince discuss...,1
4,U.S. Senate confirms Acosta to head Labor Depa...,1


### ✅ Step 2: NLTK Preprocessing Code

In [12]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\moham\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

In [15]:
def preprocess(text):
    text = text.lower()                            # Lowercase
    text = re.sub(r'<.*?>', '', text)              # Remove HTML tags
    text = re.sub(r'[^a-z\s]', '', text)           # Remove numbers and punctuation
    words = [stemmer.stem(word) for word in text.split() if word not in stop_words]
    return " ".join(words)

In [16]:
data["clean_text"] = data["text"].apply(preprocess)

In [17]:
data[["text", "clean_text"]].head()

,text,clean_text
0,Senate backs massive increase in military spen...,senat back massiv increas militari spend washi...
1,"Anonymous GOP Reps Admit Clinton Will Win, Bl...",anonym gop rep admit clinton win blame trump s...
2,‘A Gimmick’: It Just Dawned On Republicans Th...,gimmick dawn republican trump full sht wall ca...
3,USA's Tillerson and Saudi crown prince discuss...,usa tillerson saudi crown princ discuss terror...
4,U.S. Senate confirms Acosta to head Labor Depa...,us senat confirm acosta head labor depart wash...


### ✅ Step 3: TF-IDF Vectorization

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))

In [20]:
X = vectorizer.fit_transform(data["clean_text"])
y = data["label"]

In [21]:
print("TF-IDF matrix shape:", X.shape)

TF-IDF matrix shape: (44898, 5000)


### ✅ Step 4: Train Multiple Models for Fake News Detection

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

##### ✅ Train Logistic Regression

In [24]:
from sklearn.linear_model import LogisticRegression

In [25]:
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)
lr_preds = lr_model.predict(X_test)

##### ✅ Train SVM (Linear)

In [26]:
from sklearn.svm import LinearSVC

In [27]:
svm_model = LinearSVC()
svm_model.fit(X_train, y_train)
svm_preds = svm_model.predict(X_test)